In [2]:
# imports

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [5]:
load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

In [6]:
openai = OpenAI()

In [7]:
system_message = "You are an assistant that improves readability of code by just adding comments to it. "
system_message += "You cannot alter or modify the exiting code. Just add comments to the code. "

In [55]:
def user_prompt_for(code_text):
    user_prompt = "Add comments to this code to improve readability of it. Don't alter or modify anything in code.Just add comments to it."
    user_prompt += "You do not need to add comments on each line of code.Add comments on block of code and where it looks necessary."
    user_prompt += code_text
    return user_prompt

In [57]:
python_code = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [56]:
def build_message(code_text):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(code_text)}
    ]

In [15]:
def run_model(code_text, MODEL):
    completion = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=build_message(code_text),
    )
    return completion.choices[0].message.content

In [ ]:
Markdown_text="""
    <div style="text-align: center; font-size: 24px; color: #FF5733;">
        <strong> Utility to Add Comments to your Code!</strong>
    </div>
    """
HTML_MARKDOWN="""
    <div style="text-align: center; font-size: 32px; color: grey; font-weight: bold;">
        Utility to Add Comments to your Code!
    </div>
    """


In [58]:
with gr.Blocks(css=".gradio-container {background-color: black}",theme=gr.themes.Monochrome()) as ui:
    #gr.Markdown(Markdown_text)
    gr.HTML(HTML_MARKDOWN)
    with gr.Row():
        code_text = gr.Textbox(label="Code Snippet:", lines=10, value=python_code)
        result = gr.Textbox(label="Result :", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT Mini", "Claude"], label="Select model", value="GPT Mini")
        convert = gr.Button("Run")

    convert.click(run_model, inputs=[code_text, model], outputs=[result])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7882
* To create a public link, set `share=True` in `launch()`.
